<a href="https://colab.research.google.com/github/shobhawati07/MIS-report-excel-/blob/main/RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go

import plotly.express as px

Lets import the dataset ,

**INTRODUCTION**:This set has various columns customerID,transaction amount ,product infromation,orderID ans location

In [32]:
df = pd.read_csv("rfm_data.csv")
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


**Prepare the data**

In [33]:
## lets check our data is null or not
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [35]:

## Convert date object to date time
from datetime import datetime
df["PurchaseDate"]=pd.to_datetime(df['PurchaseDate'])



In [36]:
df["PurchaseDate"].max()

Timestamp('2023-06-10 00:00:00')

RFM (Recency, Frequency, Monetary) analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups. RFM helps divide customers into various categories or clusters to identify customers who are more likely to respond to promotions and also for future personalization services.

**RECENCY** (R): Days since last purchase

**FREQUENCY** (F): Total number of purchases

**MONETARY VALUE**(M): Total money this customer spent.

We will create those 3 customer attributes for each customer.

In [37]:
df["PurchaseDate"].max()

Timestamp('2023-06-10 00:00:00')

In [38]:
##Calculate the frequency for each customer:
Frequency=df.groupby(["CustomerID"])["OrderID"].count()
Frequency=Frequency.to_frame()
Frequency.rename(columns={"OrderID":"frequency"},inplace=True)

In [39]:
#Calculate the monetary for each customer:
Monetry= df.groupby(["CustomerID"])["TransactionAmount"].sum()
Monetry=pd.DataFrame(Monetry)
Monetry.rename(columns={"TransactionAmount":"monetry"},inplace=True)

In [40]:
df=df.merge(Monetry,on='CustomerID', how='left')

In [41]:
df= df.merge(Frequency,on='CustomerID',how='left')

In [42]:
#calculate recency for each customer:
df['Recenecy']=(datetime.now().date() - df['PurchaseDate'].dt.date).dt.days
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,monetry,frequency,Recenecy
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,943.31,1,125
1,2188,2023-04-11,463.70,Product A,176819,London,463.70,1,125
2,4608,2023-04-11,80.28,Product A,340062,New York,80.28,1,125
3,2559,2023-04-11,221.29,Product A,239145,London,221.29,1,125
4,9482,2023-04-11,739.56,Product A,194545,Paris,739.56,1,125
...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,759.62,1,65
996,6669,2023-06-10,941.50,Product C,987025,New York,941.50,1,65
997,8836,2023-06-10,545.36,Product C,512842,London,545.36,1,65
998,1440,2023-06-10,729.94,Product B,559753,Paris,729.94,1,65


In [43]:
rfm_df=df[["CustomerID","monetry","frequency","Recenecy"]]
rfm_df

,CustomerID,monetry,frequency,Recenecy
0,8814,943.31,1,125
1,2188,463.70,1,125
2,4608,80.28,1,125
3,2559,221.29,1,125
4,9482,739.56,1,125
...,...,...,...,...
995,2970,759.62,1,65
996,6669,941.50,1,65
997,8836,545.36,1,65
998,1440,729.94,1,65


**Customer segments with RFM Model**

The simplest way to create customers segments from RFM Model is to use Quartiles. We assign a score from 1 to 4 to Recency, Frequency and Monetary. Four is the best/highest value, and one is the lowest/worst value. A final RFM score is calculated simply by combining individual RFM score numbers.

Note: Quintiles (score from 1-5) offer better granularity, in case the business needs that but it will be more challenging to create segments since we will have 555 possible combinations. So, we will use quartiles.

In [44]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,CustomerID,monetry,frequency,Recenecy
0.25,3273.00,283.9250,1.0,80.0
0.50,5538.00,566.7100,1.0,97.0
0.75,7821.75,805.3725,1.0,110.0


In [45]:
quantiles.to_dict()

{'CustomerID': {0.25: 3273.0, 0.5: 5538.0, 0.75: 7821.75},
 'monetry': {0.25: 283.925, 0.5: 566.71, 0.75: 805.3725},
 'frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'Recenecy': {0.25: 80.0, 0.5: 97.0, 0.75: 110.0}}

**Creation of RFM Segments**

We will create two segmentation classes since, high recency is bad, while high frequency and monetary value is good.

In [46]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1
 # Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

In [47]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recenecy'].apply(RScore, args=('Recenecy',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['frequency'].apply(FMScore, args=('frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['monetry'].apply(FMScore, args=('monetry',quantiles,))

<ipython-input-47-3fa507b8f77c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm_segmentation['R_Quartile'] = rfm_segmentation['Recenecy'].apply(RScore, args=('Recenecy',quantiles,))
<ipython-input-47-3fa507b8f77c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm_segmentation['F_Quartile'] = rfm_segmentation['frequency'].apply(FMScore, args=('frequency',quantiles,))
<ipython-input-47-3fa507b8f77c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [48]:
rfm_segmentation


,CustomerID,monetry,frequency,Recenecy,R_Quartile,F_Quartile,M_Quartile
0,8814,943.31,1,125,1,1,4
1,2188,463.70,1,125,1,1,2
2,4608,80.28,1,125,1,1,1
3,2559,221.29,1,125,1,1,1
4,9482,739.56,1,125,1,1,3
...,...,...,...,...,...,...,...
995,2970,759.62,1,65,4,1,3
996,6669,941.50,1,65,4,1,4
997,8836,545.36,1,65,4,1,2
998,1440,729.94,1,65,4,1,3


To get the RFM Score we need to combine the (R_Quartile, F_Quartile,M_Quartile) together.

In [49]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

<ipython-input-49-39050fd004e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \


,CustomerID,monetry,frequency,Recenecy,R_Quartile,F_Quartile,M_Quartile,RFMScore
0,8814,943.31,1,125,1,1,4,114
1,2188,463.70,1,125,1,1,2,112
2,4608,80.28,1,125,1,1,1,111
3,2559,221.29,1,125,1,1,1,111
4,9482,739.56,1,125,1,1,3,113


Best Recency score = 4: most recently purchase.

Best Frequency score = 4: most quantity purchase.

Best Monetary score = 4: spent the most.

In [50]:
rfm_segmentation.sort_values(by='Recenecy')

,CustomerID,monetry,frequency,Recenecy,R_Quartile,F_Quartile,M_Quartile,RFMScore
999,4759,804.28,1,65,4,1,3,413
981,6761,571.97,1,65,4,1,3,413
982,3213,700.65,1,65,4,1,3,413
983,2625,765.11,1,65,4,1,3,413
984,6598,768.49,1,65,4,1,3,413
...,...,...,...,...,...,...,...,...
7,6911,433.33,1,125,1,1,2,112
5,8483,375.23,1,125,1,1,2,112
6,8317,974.88,2,125,1,4,4,144
4,9482,739.56,1,125,1,1,3,113


Let's see who are our Champions (best customers).
Lets see the customer segments


We can find [here](https://www.blastanalytics.com/blog/rfm-analysis-boosts-sales) a suggestion of key segments and then we can decide which segment to consider for further study.

Note: the suggested link use the opposite valuation: 1 as highest/best score and 4 is the lowest.

In [51]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('monetry', ascending=False).head(10)

,CustomerID,monetry,frequency,Recenecy,R_Quartile,F_Quartile,M_Quartile,RFMScore
861,5557,2379.45,3,74,4,4,4,444
789,3938,1731.53,2,78,4,4,4,444
826,9328,1710.17,2,76,4,4,4,444
842,7046,1662.10,2,75,4,4,4,444
764,2526,1635.23,2,79,4,4,4,444
951,1480,1621.62,2,68,4,4,4,444
761,9514,1553.53,2,79,4,4,4,444
809,2133,1386.69,2,77,4,4,4,444
872,2133,1386.69,2,73,4,4,4,444
745,4812,1279.44,2,80,4,4,4,444


In [52]:
rfm_segmentation["rfm_segment"]=''
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='444','rfm_segment']='Best Customer'
rfm_segmentation.loc[rfm_segmentation['M_Quartile']==4,'rfm_segment'] ='Big Spenders'
rfm_segmentation.loc[rfm_segmentation['F_Quartile']==4,'rfm_segment']='Loyal Customer'
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='244','rfm_segment']='Almost Lost'
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='144','rfm_segment']='Lost Customer'
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='111','rfm_segment']='Lost  Cheap Customer'





In [53]:
print("Customer Segment:")

print("Best customer=",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Big Spenders=',len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Loyal customer=',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print('Almost Lost=',len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customer=',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customer',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))




Customer Segment:
Best customer= 20
Big Spenders= 250
Loyal customer= 105
Almost Lost= 24
Lost Customer= 20
Lost Cheap Customer 56


In [54]:
mergedata=df[['CustomerID','Location','ProductInformation']]
mergedata

,CustomerID,Location,ProductInformation
0,8814,Tokyo,Product C
1,2188,London,Product A
2,4608,New York,Product A
3,2559,London,Product A
4,9482,Paris,Product A
...,...,...,...
995,2970,London,Product B
996,6669,New York,Product C
997,8836,London,Product C
998,1440,Paris,Product B


In [68]:
rfm_segmentation=rfm_segmentation.merge(mergedata.drop_duplicates(subset=['CustomerID']),on='CustomerID',how='left')

In [69]:
rfm_segmentation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   monetry             1000 non-null   float64
 2   frequency           1000 non-null   int64  
 3   Recenecy            1000 non-null   int64  
 4   R_Quartile          1000 non-null   int64  
 5   F_Quartile          1000 non-null   int64  
 6   M_Quartile          1000 non-null   int64  
 7   RFMScore            1000 non-null   object 
 8   rfm_segment         1000 non-null   object 
 9   Location            1000 non-null   object 
 10  ProductInformation  1000 non-null   object 
dtypes: float64(1), int64(6), object(4)
memory usage: 93.8+ KB


In [70]:
rfm_segmentation

,CustomerID,monetry,frequency,Recenecy,R_Quartile,F_Quartile,M_Quartile,RFMScore,rfm_segment,Location,ProductInformation
0,8814,943.31,1,125,1,1,4,114,Big Spenders,Tokyo,Product C
1,2188,463.70,1,125,1,1,2,112,,London,Product A
2,4608,80.28,1,125,1,1,1,111,Lost Cheap Customer,New York,Product A
3,2559,221.29,1,125,1,1,1,111,Lost Cheap Customer,London,Product A
4,9482,739.56,1,125,1,1,3,113,,Paris,Product A
...,...,...,...,...,...,...,...,...,...,...,...
995,2970,759.62,1,65,4,1,3,413,,London,Product B
996,6669,941.50,1,65,4,1,4,414,Big Spenders,New York,Product C
997,8836,545.36,1,65,4,1,2,412,,London,Product C
998,1440,729.94,1,65,4,1,3,413,,Paris,Product B
